In [2]:
import numpy as np
import opt_einsum

In [3]:
#Random_play 
A=np.random.rand(4,7,9)
B=np.random.rand(10,5)
C=np.random.rand(5,4,2)
D=np.random.rand(6,8,9,2)
path_info = opt_einsum.contract_path("ijk,jl,lmq, njpq->ijknp|j", A, B, C, D)
print(path_info[1])

  Complete contraction:  ijk,jl,lmq,njpq->ijknp|j
         Naive scaling:  8
     Optimized scaling:  5
      Naive FLOP count:  4.212e+05
  Optimized FLOP count:  2.056e+05
   Theoretical speedup:  2.049e+00
  Largest intermediate:  1.944e+05 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   4              0             lmq,jl->qj                  ijk,njpq,qj->ijknp|j
   4              0           qj,njpq->jnp                      ijk,jnp->ijknp|j
   5              0         jnp,ijk->ijknp                        ijknp->ijknp|j


## ResNet34 FLOPs study

In [9]:
#First layer conv1
X=np.random.rand(128,3,112,112)
W1=np.random.rand(116,64)
W2=np.random.rand(116,3)
W3=np.random.rand(116,7)
W4=np.random.rand(116,7)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])

#Skip max-pooling, it's a negligible cost. 

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  1.509e+14
  Optimized FLOP count:  3.867e+12
   Theoretical speedup:  3.903e+01
  Largest intermediate:  1.289e+12 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rs->rhs              bshw,rt,rw,rhs->bthw|h,w
   4              0           rhs,rw->rhsw                bshw,rt,rhsw->bthw|h,w
   5           GEMM          rhsw,rt->hswt                   bshw,hswt->bthw|h,w
   5              0        hswt,bshw->bthw                        bthw->bthw|h,w


In [14]:
#Second block conv_2x
#Single layer (implicitly include downsampling)
X=np.random.rand(128,64,56,56)
W1=np.random.rand(269,64)
W2=np.random.rand(269,64)
W3=np.random.rand(269,3)
W4=np.random.rand(269,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])
#Multiply results by 6. 

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  2.294e+13
  Optimized FLOP count:  5.156e+12
   Theoretical speedup:  4.450e+00
  Largest intermediate:  8.056e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [17]:
#Third block conv_3x
#First layer downsample
X=np.random.rand(128,64,28,28)
W1=np.random.rand(372,128)
W2=np.random.rand(372,64)
W3=np.random.rand(372,3)
W4=np.random.rand(372,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  4.492e+12
  Optimized FLOP count:  6.445e+11
   Theoretical speedup:  6.969e+00
  Largest intermediate:  1.007e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [18]:
#Third Block
#Single non-downsampling layer
X=np.random.rand(128,128,28,28)
W1=np.random.rand(562,128)
W2=np.random.rand(562,128)
W3=np.random.rand(562,3)
W4=np.random.rand(562,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])
#Multiply result x7 and add to cost of the downsample layer. 

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  7.710e+12
  Optimized FLOP count:  1.289e+12
   Theoretical speedup:  5.981e+00
  Largest intermediate:  1.007e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [19]:
#Fourth block
#First layer downsample
X=np.random.rand(128,128,14,14)
W1=np.random.rand(756,256)
W2=np.random.rand(756,128)
W3=np.random.rand(756,3)
W4=np.random.rand(756,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  2.263e+12
  Optimized FLOP count:  1.614e+11
   Theoretical speedup:  1.402e+01
  Largest intermediate:  1.259e+09 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [20]:
#Fourth block conv_4x
#Single non-downsampling layer
X=np.random.rand(128,256,14,14)
W1=np.random.rand(1138,256)
W2=np.random.rand(1138,256)
W3=np.random.rand(1138,3)
W4=np.random.rand(1138,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])
#Multiply result by 11 and add to downsampling cost. 

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  5.277e+12
  Optimized FLOP count:  3.229e+11
   Theoretical speedup:  1.634e+01
  Largest intermediate:  1.259e+09 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [21]:
#Fifth block conv_5x
#First layer downsample
X=np.random.rand(128,256,7,7)
W1=np.random.rand(1524,512)
W2=np.random.rand(1524,256)
W3=np.random.rand(1524,3)
W4=np.random.rand(1524,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  2.780e+12
  Optimized FLOP count:  4.208e+10
   Theoretical speedup:  6.605e+01
  Largest intermediate:  1.574e+08 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w


In [22]:
#Fifth block conv_5x
#Single non-downsampling layer
X=np.random.rand(128,512,7,7)
W1=np.random.rand(2290,512)
W2=np.random.rand(2290,512)
W3=np.random.rand(2290,3)
W4=np.random.rand(2290,3)
path_info = opt_einsum.contract_path("bshw,rt,rs,rh,rw->bthw|h,w", X, W1, W2, W3, W4)
print(path_info[1])
#Multiply result by 5 and add to downsampling cost. 

  Complete contraction:  bshw,rt,rs,rh,rw->bthw|h,w
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  7.942e+12
  Optimized FLOP count:  8.597e+10
   Theoretical speedup:  9.238e+01
  Largest intermediate:  1.574e+08 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             rh,rt->rht              bshw,rs,rw,rht->bthw|h,w
   3              0             rw,rs->rws                bshw,rht,rws->bthw|h,w
   5           GEMM          rws,rht->wsht                   bshw,wsht->bthw|h,w
   5              0        wsht,bshw->bthw                        bthw->bthw|h,w
